In [10]:
import tensorflow as tf
from tensorflow import keras

In [6]:
import os
import time

dir_name = "my_logs"
dir_path = os.path.join(os.curdir, dir_name)
if os.path.exists(dir_path):
    pass
else:
    os.mkdir(dir_path)
    
def get_run_logdir():
    run_id = time.strftime("run_%Y_%m_%d-%H_%M_%S")
    return os.path.join(dir_path, run_id)

In [7]:
get_run_logdir()

'.\\my_logs\\run_2021_03_01-05_55_23'

In [8]:
from sklearn.datasets import fetch_california_housing
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

housing = fetch_california_housing()

x_train_full, x_test, y_train_full, y_test = train_test_split(
    housing.data, housing.target)
x_train, x_val, y_train, y_val = train_test_split(
    x_train_full, y_train_full)

scaler = StandardScaler()
x_train = scaler.fit_transform(x_train)
x_val = scaler.transform(x_val)
x_test = scaler.transform(x_test)

In [11]:
input_ = keras.layers.Input(shape=x_train.shape[1: ])
h1 = keras.layers.Dense(30, activation="relu")(input_)
h2 = keras.layers.Dense(30, activation="relu")(h1)
concat = keras.layers.Concatenate()([input_, h2])
output = keras.layers.Dense(1)(concat)

model = keras.Model(inputs=[input_], outputs=[output])

In [16]:
model.compile(loss="mse", optimizer=keras.optimizers.SGD(lr=0.001))

In [17]:
checkpoint_cb = keras.callbacks.ModelCheckpoint("my_keras_model.h5", 
                                               save_best_only=True)
early_stopping_cb = keras.callbacks.EarlyStopping(patience=5,
                                                 restore_best_weights=True)
run_logdir = get_run_logdir()
tensorboard_cb = keras.callbacks.TensorBoard(run_logdir)

hist = model.fit(x_train, y_train, epochs=200,
                  validation_data=(x_val, y_val),
                callbacks=[checkpoint_cb, early_stopping_cb, tensorboard_cb])

Epoch 1/200
363/363 [==============================] - 1s 2ms/step - loss: 0.6225 - val_loss: 9.5640
Epoch 2/200
363/363 [==============================] - 0s 536us/step - loss: 0.8047 - val_loss: 37.7356
Epoch 3/200
363/363 [==============================] - 0s 533us/step - loss: 5.7527 - val_loss: 1.2236
Epoch 4/200
363/363 [==============================] - 0s 519us/step - loss: 0.6097 - val_loss: 0.5462
Epoch 5/200
363/363 [==============================] - 0s 547us/step - loss: 0.4570 - val_loss: 0.4811
Epoch 6/200
363/363 [==============================] - 0s 727us/step - loss: 0.4424 - val_loss: 0.4708
Epoch 7/200
363/363 [==============================] - 0s 528us/step - loss: 0.4471 - val_loss: 0.4608
Epoch 8/200
363/363 [==============================] - 0s 528us/step - loss: 0.4234 - val_loss: 0.4546
Epoch 9/200
363/363 [==============================] - 0s 519us/step - loss: 0.4104 - val_loss: 0.4464
Epoch 10/200
363/363 [==============================] - 0s 514us/step - lo

Epoch 80/200
363/363 [==============================] - 0s 536us/step - loss: 0.3001 - val_loss: 0.3703
Epoch 81/200
363/363 [==============================] - 0s 543us/step - loss: 0.3160 - val_loss: 0.3734
Epoch 82/200
363/363 [==============================] - 0s 535us/step - loss: 0.3113 - val_loss: 0.3699
Epoch 83/200
363/363 [==============================] - 0s 525us/step - loss: 0.3113 - val_loss: 0.3722
Epoch 84/200
363/363 [==============================] - 0s 542us/step - loss: 0.3031 - val_loss: 0.3715
Epoch 85/200
363/363 [==============================] - 0s 589us/step - loss: 0.3071 - val_loss: 0.3696
Epoch 86/200
363/363 [==============================] - 0s 531us/step - loss: 0.3164 - val_loss: 0.3704
Epoch 87/200
363/363 [==============================] - 0s 536us/step - loss: 0.3142 - val_loss: 0.3702
Epoch 88/200
363/363 [==============================] - 0s 522us/step - loss: 0.3100 - val_loss: 0.3692
Epoch 89/200
363/363 [==============================] - 0s 525us

363/363 [==============================] - 0s 594us/step - loss: 0.2852 - val_loss: 0.3529
Epoch 159/200
363/363 [==============================] - 0s 561us/step - loss: 0.2959 - val_loss: 0.3514
Epoch 160/200
363/363 [==============================] - 0s 539us/step - loss: 0.2912 - val_loss: 0.3510
Epoch 161/200
363/363 [==============================] - 0s 550us/step - loss: 0.2885 - val_loss: 0.3510
Epoch 162/200
363/363 [==============================] - 0s 586us/step - loss: 0.2818 - val_loss: 0.3509
Epoch 163/200
363/363 [==============================] - 0s 531us/step - loss: 0.2853 - val_loss: 0.3505
Epoch 164/200
363/363 [==============================] - 0s 522us/step - loss: 0.2825 - val_loss: 0.3521
Epoch 165/200
363/363 [==============================] - 0s 517us/step - loss: 0.2937 - val_loss: 0.3509
Epoch 166/200
363/363 [==============================] - 0s 522us/step - loss: 0.2841 - val_loss: 0.3511
Epoch 167/200
363/363 [==============================] - 0s 526us/ste

In [23]:
import numpy as np

test_logdir = get_run_logdir()
writer = tf.summary.create_file_writer(test_logdir)
with writer.as_default():
    for step in range(1, 1000+1):
        tf.summary.scalar("my_scalar", np.sin(step/10), step=step)
        
        data = (np.random.randn(100)+2) * step/100
        tf.summary.histogram("my_hist", data, buckets=50, step=step)
        
        images = np.random.rand(2, 32, 32, 3)
        tf.summary.image("my_images", images*step/1000, step=step)
        
        texts = ["The step is " + str(step), "Its square is ", str(step**2)]
        tf.summary.text("my_text", texts, step=step)
        
        sine_wave = tf.math.sin(tf.range(12000)/48000 * 2 * np.pi * step)
        audio = tf.reshape(tf.cast(sine_wave, tf.float32), [1, -1, 1])
        tf.summary.audio("my_audio", audio, sample_rate=24000, step=step)

**tf.summary** 패키지를 이용하여 저수준에서 텐서보드 로그를 만드는 방법.  
스칼라, 히스토그램, 이미지, 오디오, 텍스트 등 여러가지 정보를 로그로 만들 수 있음.

In [22]:
np.random.randn(100)

array([ 2.45020635e+00, -1.68286334e+00, -8.74575628e-01, -9.32650703e-01,
       -1.75834461e-01, -6.26521368e-01,  1.33874202e+00,  1.51790038e+00,
       -6.61331878e-01, -2.60644333e-01, -7.97762275e-01,  2.29858851e-03,
        4.09361961e-01, -5.56289943e-01, -5.69480478e-01, -9.06457907e-01,
        5.40825196e-03, -1.92204382e+00,  5.63634239e-01, -7.15341513e-02,
       -7.07368860e-01,  1.94188080e+00, -1.00360124e-01,  9.04700066e-01,
       -1.11180206e+00,  1.88480607e-01,  2.30865430e-01, -8.59603288e-01,
        5.26812150e-01,  6.11759908e-01, -4.07466879e-01,  8.82322750e-01,
        6.72408317e-01, -2.50126261e-01, -4.91759691e-01,  1.30303009e+00,
       -2.06822843e-01, -8.50266819e-02, -3.74476851e-01, -3.12461552e-02,
       -3.99829039e-01, -7.01129160e-01,  1.48378973e+00,  6.11516304e-01,
       -3.21657506e-01,  1.00909510e+00, -6.21692627e-01,  4.82294205e-01,
       -6.70475780e-01,  7.96829017e-01, -9.47730628e-01, -3.72041056e-01,
        7.82253820e-01,  